In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

from PIL import Image

In [ ]:
#8 MIN TO RUN
def load_data():
    # Load training images and labels
    train_images = []
    train_labels = []
    train_dir = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/train/images'
    train_label_file = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/train/labels'
    for filename in os.listdir(train_dir):
        img = Image.open(os.path.join(train_dir, filename))
        img = img.resize((224, 224)) # Resize to a common size
        img_arr = np.array(img)
        train_images.append(img_arr)
        
        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_file_path = os.path.join(train_label_file, label_filename)
        with open(label_file_path, 'r') as f:
            text = f.read().strip()
            # Map "no_mask" to 0, "mask" to 1, and "improper_mask" to 2
            if text == "1":
                label = 1
            elif text == "0":
                label = 0
            elif text == "2":
                label = 2
            train_labels.append(label)

    # Load testing images and labels
    test_images = []
    test_labels = []
    test_dir = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/test/images'
    test_label_dir = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/test/labels'
    for filename in os.listdir(test_dir):
        img = Image.open(os.path.join(test_dir, filename))
        img = img.resize((224, 224)) # Resize to a common size
        img_arr = np.array(img)
        test_images.append(img_arr)
        
        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_file_path = os.path.join(test_label_dir, label_filename)
        with open(label_file_path, 'r') as f:
            text = f.read().strip()
            # Map "no_mask" to 0, "mask" to 1, and "improper_mask" to 2
            if text == "1":
                label = 1
            elif text == "0":
                label = 0
            elif text == "2":
                label = 2
            test_labels.append(label)

    # Convert the lists to numpy arrays
    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)
    
    return train_images, test_images, train_labels, test_labels

# Load the dataset
X_train, X_test, y_train, y_test = load_data()

print("Number of training images: ", len(X_train))
print("Number of training labels: ", len(y_train))
print("Number of testing images: ", len(X_test))
print("Number of testing labels: ", len(y_test))

print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test: ", y_test.shape)

# Print information about the dataset
print("Number of training images: ", X_train.shape[0])
print("Number of training labels: ", y_train.shape[0])
print("Number of testing images: ", X_test.shape[0])
print("Number of testing labels: ", y_test.shape[0])

Number of training images:  2394
Number of training labels:  2394
Number of testing images:  458
Number of testing labels:  458
Shape of X_train:  (2394, 224, 224, 3)
Shape of y_train:  (2394,)
Shape of X_test:  (458, 224, 224, 3)
Shape of y_test:  (458,)
Number of training images:  2394
Number of training labels:  2394
Number of testing images:  458
Number of testing labels:  458


In [ ]:
# TIME == 1 MIN
from skimage.feature import hog
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler

# Define a function to extract HOG features from images
def extract_hog_features(images):
    hog_features = []
    for image in images:
        if image.ndim == 2:
            hog_feature = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=False)
        else:
            hog_feature = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=False, multichannel=True, channel_axis=2)
        hog_features.append(hog_feature)
    return np.array(hog_features)


# Extract HOG features from the training and testing images
X_train_hog = extract_hog_features(X_train)
X_test_hog = extract_hog_features(X_test)

# Standardize the data
scaler = StandardScaler()
X_train_hog = scaler.fit_transform(X_train_hog)
X_test_hog = scaler.transform(X_test_hog)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'hidden_layer_sizes': [(100,), (50, 100, 50)], 
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']
}

# Train an MLP model with hyperparameter tuning
mlp = MLPClassifier(max_iter=500, random_state=42, early_stopping=True)
grid_search = GridSearchCV(mlp, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train_hog, y_train)

# Evaluate the best model on the testing set
best_mlp = grid_search.best_estimator_
y_pred = best_mlp.predict(X_test_hog)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print("Best hyperparameters:", grid_search.best_params_)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)
print("Confusion matrix:\n", conf_matrix)


<ipython-input-6-4f405d479f4d>:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  hog_feature = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=False, multichannel=True, channel_axis=2)


Best hyperparameters: {'alpha': 0.01, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'constant'}
Accuracy: 0.8624454148471615
Precision: 0.8666134811331317
Recall: 0.8624454148471615
F1 score: 0.8604566219096814
Confusion matrix:
 [[ 30  20   1]
 [ 28 359   1]
 [  2  11   6]]


In [ ]:
import joblib
import os

# Define directory path to save the joblib file
directory = '/content/drive/MyDrive/CW_Folder_UG/Models'

# Create directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Define file path to save the joblib file
filepath = os.path.join(directory, 'hog-mlp.joblib')

# Dump the object to the joblib file
joblib.dump(best_mlp, filepath)

['/content/drive/MyDrive/CW_Folder_UG/Models/hog-mlp.joblib']

In [ ]:
import joblib
import os

# create folder if it doesn't exist
folder_path = os.path.join('content', 'drive', 'MyDrive', 'CW_Folder_UG', 'Models')
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# save model in folder
file_path = os.path.join(folder_path, 'hog-mlp.joblib')
joblib.dump(best_mlp, file_path)



['content/drive/MyDrive/CW_Folder_UG/Models/hog-mlp.joblib']